In [35]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
from dotenv import load_dotenv
import os
import time
from botocore.exceptions import NoCredentialsError, PartialCredentialsError


# Load environment variables from a .env file
load_dotenv()
# Retrieve the environment variables
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_session_token = os.getenv('AWS_SESSION_TOKEN')
region_name = os.getenv('AWS_REGION')

# Initialize a session using Amazon DynamoDB
session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    aws_session_token=aws_session_token,
    region_name=region_name
)

dynamodb = session.resource('dynamodb')
print("CONEXION EXITOSA")



CONEXION EXITOSA


In [37]:
# Función para crear tablas
def create_tables():
    try:
        for table_name, pk in [("Usuarios", "UserID"), ("Productos", "ProductoID"), ("Pedidos", "PedidoID")]:
            table = dynamodb.create_table(
                TableName=table_name,
                KeySchema=[{"AttributeName": pk, "KeyType": "HASH"}],
                AttributeDefinitions=[{"AttributeName": pk, "AttributeType": "S"}],
                ProvisionedThroughput={"ReadCapacityUnits": 5, "WriteCapacityUnits": 5}
            )
            table.wait_until_exists()
            print(f"Tabla {table_name} creada correctamente")
    except Exception as e:
        print(f"Error: {e}")

create_tables()

Tabla Usuarios creada correctamente
Tabla Productos creada correctamente
Tabla Pedidos creada correctamente


In [38]:
# Función para insertar datos
def insert_data():
    try:
        data = [
            ("Usuarios", {"UserID": "U1", "Nombre": "Alice"}),
            ("Productos", {"ProductoID": "P1", "Nombre": "Laptop"}),
            ("Pedidos", {"PedidoID": "O1", "Cliente": "Alice"})
        ]
        
        for table_name, item in data:
            table = dynamodb.Table(table_name)
            # Inserta el ítem
            table.put_item(Item=item)
            print(f"Datos insertados en la tabla {table_name}: {item}")
            
            # Verificación de la inserción
            response = table.get_item(Key={list(item.keys())[0]: list(item.values())[0]})
            if 'Item' in response:
                print(f"Verificación de inserción exitosa en {table_name}: {response['Item']}")
            else:
                print(f"No se encontró el registro insertado en {table_name}.")
        
        print("Todos los datos fueron insertados correctamente.")
    except Exception as e:
        print(f"Error al insertar datos: {e}")
insert_data()

Datos insertados en la tabla Usuarios: {'UserID': 'U1', 'Nombre': 'Alice'}
Verificación de inserción exitosa en Usuarios: {'Nombre': 'Alice', 'UserID': 'U1'}
Datos insertados en la tabla Productos: {'ProductoID': 'P1', 'Nombre': 'Laptop'}
Verificación de inserción exitosa en Productos: {'Nombre': 'Laptop', 'ProductoID': 'P1'}
Datos insertados en la tabla Pedidos: {'PedidoID': 'O1', 'Cliente': 'Alice'}
Verificación de inserción exitosa en Pedidos: {'Cliente': 'Alice', 'PedidoID': 'O1'}
Todos los datos fueron insertados correctamente.


In [41]:
def obtener_registros():
    try:
        for table_name in ["Usuarios", "Productos", "Pedidos"]:
            table = dynamodb.Table(table_name)
            response = table.scan()
            items = response.get('Items', [])
            print(f"Registros en la tabla {table_name}: {items}")
    except Exception as e:
        print(f"Error al obtener registros: {e}")

obtener_registros()

Registros en la tabla Usuarios: [{'Nombre': 'Alice Updated', 'UserID': 'U1'}]
Registros en la tabla Productos: [{'Nombre': 'Laptop Updated', 'ProductoID': 'P1'}]
Registros en la tabla Pedidos: [{'Cliente': 'Alice Updated', 'PedidoID': 'O1'}]


In [40]:
def actualizar_registros():
    try:
        updates = [
            ("Usuarios", {"UserID": "U1"}, "SET Nombre = :nombre", {":nombre": "Alice Updated"}),
            ("Productos", {"ProductoID": "P1"}, "SET Nombre = :nombre", {":nombre": "Laptop Updated"}),
            ("Pedidos", {"PedidoID": "O1"}, "SET Cliente = :cliente", {":cliente": "Alice Updated"})
        ]
        
        for table_name, key, update_expression, expression_attribute_values in updates:
            table = dynamodb.Table(table_name)
            # Obtener el registro antes de la actualización
            response_before = table.get_item(Key=key)
            item_before = response_before.get('Item', {})
            
            # Actualizar el registro
            table.update_item(
                Key=key,
                UpdateExpression=update_expression,
                ExpressionAttributeValues=expression_attribute_values
            )
            print(f"Registro actualizado en la tabla {table_name}: {key}")
            
            # Obtener el registro después de la actualización
            response_after = table.get_item(Key=key)
            item_after = response_after.get('Item', {})
            
            # Mostrar los registros antes y después de la actualización
            print(f"Registro antes de la actualización en {table_name}: {item_before}")
            print(f"Registro después de la actualización en {table_name}: {item_after}")
        
        print("Todos los registros fueron actualizados correctamente.")
    except Exception as e:
        print(f"Error al actualizar registros: {e}")

actualizar_registros()

Registro actualizado en la tabla Usuarios: {'UserID': 'U1'}
Registro actualizado en la tabla Productos: {'ProductoID': 'P1'}
Registro actualizado en la tabla Pedidos: {'PedidoID': 'O1'}
Todos los registros fueron actualizados correctamente.


In [44]:
# Verificar eliminación de registros
def verificar_eliminacion():
    try:
        for table_name, key in [("Usuarios", {"UserID": "U1"}), ("Productos", {"ProductoID": "P1"}), ("Pedidos", {"PedidoID": "O1"})]:
            table = dynamodb.Table(table_name)
            response = table.get_item(Key=key)
            if 'Item' in response:
                print(f"El registro en la tabla {table_name} no se eliminó: {response['Item']}")
            else:
                print(f"El registro en la tabla {table_name} fue eliminado correctamente.")
    except Exception as e:
        print(f"Error al verificar eliminación de registros: {e}")

verificar_eliminacion()

El registro en la tabla Usuarios fue eliminado correctamente.
El registro en la tabla Productos fue eliminado correctamente.
El registro en la tabla Pedidos fue eliminado correctamente.


In [45]:
obtener_registros()

Registros en la tabla Usuarios: []
Registros en la tabla Productos: []
Registros en la tabla Pedidos: []
